In [79]:
import pandas as pd
import tweepy #https://github.com/tweepy/tweepy
import csv
from datetime import datetime
import os
import re
import requests
import twitter
import lxml.html
import time

In [80]:
ironhack= pd.read_csv('ironhack_twitter_community.csv')
ironhack.describe()
ironhack_q3 = ironhack[(ironhack['protected']==False)&(ironhack['statuses_count']>=793)&(ironhack['statuses_count']<=3307)]
ironhack_q3.shape

(2890, 14)

In [81]:
testers = pd.read_csv('ironhack_personality_testers.csv')

In [82]:
#Twitter API credentials
consumerkey = 'dANUadRrWFXEpWa2mnobFSoIn'
consumersecret = 'wVOQVveDmnnJWkvDuuU5n5d1ZCXvaD67RzvYf9ZwuGSb76GCQt'
accesskey = '1239898689797324801-4lPKrgttoF3XuRuQNbLmRNzBDzLHGw'
accesssecret = 'ChAbjrZunFfcFher9JTtZP3uzw7FeAogXpelcV12yFwBD'

auth = tweepy.OAuthHandler(consumerkey, consumersecret)
auth.set_access_token(accesskey, accesssecret)
api = tweepy.API(auth)


In [83]:
t = twitter.Api(
    consumer_key = consumerkey,
    consumer_secret = consumersecret,
    access_token_key = accesskey, 
    access_token_secret = accesssecret,
    tweet_mode='extended', 
    sleep_on_rate_limit=True# this ensures that we get the full text of the users' original tweets
)

In [84]:
def first_200(name): #working
    sample = t.GetUserTimeline(screen_name=name, count=200)
    return sample

text = ''
hashtags = []
count = 0
def testing_first_200(name): #working
    global hashtags
    global text
    global count
    count +=1
    sample = t.GetUserTimeline(screen_name=name, count=200)
    print('user timeline', name)
    text = ''
    hashtags = []
    for n,tweet in enumerate(sample):
        time.sleep(1)
        text += tweet.full_text
        #isempty
        hashtags.append(clean_hashtags(tweet.hashtags))
 
    print(count,' user scanned')
        

    output = [text, hashtags]   
    return output

def get_tweets(name):
    all_tweets = []
    sample = first_200(name)
    all_tweets.extend(sample)
    last_id = sample[-1].id
    time.sleep(1)
    for i in range(1):
        new = t.GetUserTimeline(screen_name=name, max_id=last_id-1)
        if len(new) > 0:
            all_tweets.extend(new)
            last_id = new[-1].id
        else:
            break
    
    return all_tweets

def clean_hashtags(hashtags):
    """
    Turns data with any number of hashtags like this - [Hashtag(Text='STEMonStation')] - to a list like this -
    ['STEMonStation']
    """
    cleaned = []
    if len(hashtags) >= 1:
        for i in range(len(hashtags)):
            cleaned.append(hashtags[i].text)        
    return cleaned


def full_tweet_text_hashtag(name) : 
    tweets_expanded = get_tweets(name)
    
    text = ''
    hashtags = []
    
    for tweet in tweets_expanded :
            text += tweet.full_text
            hashtags.append(clean_hashtags(tweet.hashtags))
    output = [text, hashtags]
            
    return output


In [85]:
#testers['apply_function_test'] = testers['screen_name'].apply(testing_first_200)

In [86]:
#testers['apply_function_test'][0]

In [68]:
ironhack_q3['apply_function_test'] = ironhack_q3['screen_name'].apply(testing_first_200)

user timeline estebanquirogas


KeyboardInterrupt: 

In [88]:
testing_first_200('WHO')

KeyboardInterrupt: 

In [33]:
testers.to_csv('full_testers.csv')
